# Esto es código auxiliar para seguir el siguiente proceso de Business Inteligence:
<img src='images/proceso.png'>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/Amazon-Products.csv", index_col=0)
df.pop('image')
df.pop('link')
df = df.set_index('name')

In [3]:
df.head()

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
name,,,,,,
"Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1 Convertible, Copper, Anti-Viral + Pm 2.5 Filter, 2023 Model, White, Gls18I3...",appliances,Air Conditioners,4.2,"2,255","₹32,999","₹58,990"
"LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.2,"2,948","₹46,490","₹75,990"
"LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Copper, Super Convertible 6-In-1 Cooling, Hd Filter With Anti Virus Protection,...",appliances,Air Conditioners,4.2,"1,206","₹34,490","₹61,990"
"LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (Copper, Super Convertible 6-in-1 Cooling, HD Filter with Anti-Virus Protectio...",appliances,Air Conditioners,4.0,69,"₹37,990","₹68,990"
"Carrier 1.5 Ton 3 Star Inverter Split AC (Copper,ESTER Dxi, 4-in-1 Flexicool Inverter, 2022 Model,R32,White)",appliances,Air Conditioners,4.1,630,"₹34,490","₹67,790"


In [4]:
null_percentages = pd.DataFrame(100 * df.isnull().sum() / len(df)).transpose()
null_percentages

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
0,0.0,0.0,31.8707,31.8707,11.08859,3.229421


Tenemos más de un 31% de ragings a null.<br>
Consultando el kaggle original, vemos que esto indica que no ha habido ningún rating de ese producto.<br>



De aquí ya extraemos la conclusión de que en torno al 30% de los productos de amazon no llegan a tener unas ventas significativas y se quedan en un intento muy vago sin ni siquiera una opinión.

Así sabemos que no estamos perdiendo información que nos interese si eliminamos todas las filas que tengan el rating a null.<br>
Tras hacerlo, tenemos la siguiente tabla de porcentajes nulos:

In [5]:
df.dropna(inplace=True)
null_percentages = pd.DataFrame(100 * df.isnull().sum() / len(df)).transpose()
null_percentages

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
0,0.0,0.0,0.0,0.0,0.0,0.0


Ahora damos el tipo correspondiente a cada columna

In [6]:
# Dar formato de número a los ratings y no_of_ratings
df['no_of_ratings'] = pd.to_numeric(df['no_of_ratings'], errors='coerce')
df.dropna(subset=['no_of_ratings'], inplace=True)
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df.dropna(subset=['ratings'], inplace=True)

# Convertimos de rupias a dólares
from forex_python.converter import CurrencyRates

c = CurrencyRates()
tasa_conversion = c.get_rate('INR', 'USD')

df['discount_price'] = df['discount_price'].apply(lambda x: x[1:]) # quitamos el símbolo de rupias
df['discount_price'] = pd.to_numeric(df['discount_price'], errors='coerce')
df.dropna(subset=['discount_price'], inplace=True)
df['discount_price'] = df['discount_price'] * tasa_conversion

df['actual_price'] = df['actual_price'].apply(lambda x: x[1:]) # quitamos el símbolo de rupias
df['actual_price'] = pd.to_numeric(df['actual_price'], errors='coerce')
df.dropna(subset=['actual_price'], inplace=True)
df['actual_price'] = df['actual_price'] * tasa_conversion



df.describe()

,ratings,no_of_ratings,discount_price,actual_price
count,112603.000000,112603.000000,112603.000000,112603.000000
mean,3.775243,88.874231,4.510466,8.800893
std,0.769010,169.951679,2.045108,3.057014
min,1.000000,1.000000,0.097893,0.122367
25%,3.400000,4.000000,3.046933,6.106102
50%,3.900000,16.000000,4.221654,9.165271
75%,4.200000,80.000000,5.812421,12.224440
max,5.000000,999.000000,12.212203,12.236554


Tras borrar todas las columnas nos quedamos con algo más de 100k; para un fin educativo es suficiente

In [7]:
df.head()

,main_category,sub_category,ratings,no_of_ratings,discount_price,actual_price
name,,,,,,
AC Stand/Heavy Duty Air Conditioner Outdoor Unit Mounting Bracket,appliances,Air Conditioners,5.0,2.0,7.329769,12.224440
RISARYA H20R1203 IGBT 1PCS POWER TRANSISTOR 20A 1200V REVERSE CONDUCTING HEATING AND WIDELY USED IN INDUCTION COOKER T0247...,appliances,Air Conditioners,4.0,4.0,2.435099,6.106102
Glun Multipurpose Portable Electronic Digital Weighing Scale Weight Machine (10 Kg - with Back Light),appliances,All Appliances,3.8,365.0,2.435099,11.000772
"Tosaa 18 Pcs - 3 Different Size Plastic Food Snack Bag Pouch Clip Sealer Large,Medium,Small Plastic Snack Seal Sealing Bag...",appliances,All Appliances,3.9,82.0,0.721964,1.211431
Finalize Tap Water Purifier Filter Faucet 6 Layer Carbon Activated Dust Chlorine Remover Water Softener for Drinking Cartr...,appliances,All Appliances,4.1,77.0,2.557465,12.224440


Procedemos a pasar las tablas a una base de datos SQLite con la siguiente estructura en estrella:

<img src="images/tablas_hechos.png">

Lo primero es crear las tablas:

In [8]:
categorias = pd.concat([df['sub_category'], df['main_category']], axis=1).groupby(['sub_category']).sum().copy()
categorias

,main_category
sub_category,
Air Conditioners,appliancesappliances
All Appliances,appliancesappliancesappliancesappliancesapplia...
All Car & Motorbike Products,car & motorbikecar & motorbikecar & motorbikec...
All Electronics,"tv, audio & camerastv, audio & camerastv, audi..."
All Exercise & Fitness,sports & fitnesssports & fitnesssports & fitne...
...,...
Washing Machines,appliancesappliancesappliancesappliancesapplia...
Watches,accessoriesaccessoriesaccessoriesaccessoriesac...
Western Wear,women's clothingwomen's clothingwomen's clothi...


In [11]:
ventas = df.copy()
ventas.pop('main_category')
ventas.head()

,sub_category,ratings,no_of_ratings,discount_price,actual_price
name,,,,,
AC Stand/Heavy Duty Air Conditioner Outdoor Unit Mounting Bracket,Air Conditioners,5.0,2.0,7.329769,12.224440
RISARYA H20R1203 IGBT 1PCS POWER TRANSISTOR 20A 1200V REVERSE CONDUCTING HEATING AND WIDELY USED IN INDUCTION COOKER T0247...,Air Conditioners,4.0,4.0,2.435099,6.106102
Glun Multipurpose Portable Electronic Digital Weighing Scale Weight Machine (10 Kg - with Back Light),All Appliances,3.8,365.0,2.435099,11.000772
"Tosaa 18 Pcs - 3 Different Size Plastic Food Snack Bag Pouch Clip Sealer Large,Medium,Small Plastic Snack Seal Sealing Bag...",All Appliances,3.9,82.0,0.721964,1.211431
Finalize Tap Water Purifier Filter Faucet 6 Layer Carbon Activated Dust Chlorine Remover Water Softener for Drinking Cartr...,All Appliances,4.1,77.0,2.557465,12.224440


# Ya tenemos los datos bien estructurados, solamente nos queda añadirlos a una base de datos:

In [12]:
import sqlite3

# Creamos una conexión a la base de datos SQLite
conn = sqlite3.connect('hechos.db')

# Pasamos los DataFrames a la base de datos
categorias.to_sql('categorias', conn, if_exists='replace', index=True)
ventas.to_sql('ventas', conn, if_exists='replace', index=True, 
           dtype={'name': 'INT', 'sub_category': 'TEXT',
                  'FOREIGN KEY(subcategory)': 'REFERENCES categorias(subcategorias)',
                  'ratings': 'FLOAT', 'no_of_ratings': 'FLOAT',
                  'discount_price': 'FLOAT', 'actual_price': 'FLOAT'})

# Cerramos la conexión a la base de datos
conn.close()